<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 3: прогнозирование временных рядов</center></h2>

**Мягкий дедлайн:** 23:59MSK 19.05.2024

**Жесткий дедлайн:** 23:59MSK 26.05.2024

In [ ]:
!pip install -r requirements.txt

In [ ]:
import warnings
import gdown
import pandas as pd
import numpy as np
from copy import deepcopy

from etna.analysis import cross_corr_plot
from etna.analysis import distribution_plot
from etna.analysis import plot_anomalies
from etna.analysis import plot_anomalies_interactive
from etna.analysis import plot_backtest
from etna.analysis import plot_correlation_matrix
from etna.analysis import plot_forecast
from etna.analysis import acf_plot
from etna.analysis import stl_plot
from etna.analysis.outliers import get_anomalies_density
from etna.analysis.outliers import get_anomalies_hist
from etna.analysis.outliers import get_anomalies_median
from etna.analysis.outliers import get_anomalies_prediction_interval
from etna.datasets import TSDataset
from etna.ensembles import DirectEnsemble, StackingEnsemble, VotingEnsemble
from etna.metrics import MAE, MAPE, MSE, SMAPE
from etna.models import CatBoostPerSegmentModel
from etna.models import CatBoostMultiSegmentModel
from etna.models import LinearPerSegmentModel
from etna.models import MovingAverageModel
from etna.models import NaiveModel
from etna.models import AutoARIMAModel
from etna.models import ProphetModel
from etna.models import SeasonalMovingAverageModel
from etna.pipeline import AutoRegressivePipeline, Pipeline, assemble_pipelines
from etna.transforms import DateFlagsTransform
from etna.transforms import FilterFeaturesTransform
from etna.transforms import HolidayTransform
from etna.transforms import LagTransform
from etna.transforms import LinearTrendTransform
from etna.transforms import LogTransform
from etna.transforms import MeanTransform
from etna.transforms import MedianOutliersTransform
from etna.transforms import SegmentEncoderTransform
from etna.transforms import TimeSeriesImputerTransform
from etna.datasets import load_dataset

warnings.filterwarnings("ignore")

В данной задаче мы будем решать одну из актуальных практических задач, с которыми, в том числе, сталкиваются разработчики ETNA - прогнозирование объема необходимой наличности в банкоматах. Пожалуй, ни для кого не секрет, что отсутствие необходимой суммы в банкомате не делает клиента банка счастливее. В то же время, избыток заправленной в банкомат наличности приводит к упущенной выгоде - ведь, в конечном счете, эти деньги могли быть размещены в качестве краткосрочного депозита на межбанковском рынке. Для кредитной организации с обширной сетью банкоматов даже незначительное улучшение качества предсказания способно внести значительный вклад в прибыльность этой ветки бизнеса.

В качестве инструмента воспользуемся библиотекой ETNA, документацию можно прочитать [тут](https://docs.etna.ai/stable/), и [чат](https://t.me/etna_support) комьюнити.

Мы будем использовать дневные данные реальных банкоматов для чего возьмем мультисегментный датасет.

In [ ]:
url = 'https://gist.githubusercontent.com/Polzovat123/4d6d5e544e93429c2d3db29196e9c918/raw/24b27e60f587128d71678e267f51cd48472c6a84/atms_daily_cash.csv'
output = 'atms_daily_cash.csv'
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

df = pd.read_csv('atms_daily_cash.csv', index_col=False)

Полученные данные были просуммированы по банкоматам и дням эксплуатации, после чего залиты в gist, откуда мы и берем их.

Взглянем на то, что представляют из себя данные после этих действий:

In [ ]:
df.head()

### Задание 1. EDA (1.75 балл)

Проведите разведывательный анализ данных с помощью `ETNA`- постройте графики, на которых можно судить о наличии тренда, сезонности и прочих зависимостей. Проанализируйте корреляции. Какие выводы вы можете сделать?

Ссылку на туториал по EDA можно найти [тут](https://docs.etna.ai/stable/tutorials/103-EDA.html#EDA).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
sns.set()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
sns.histplot(data=df['operation_value'], palette = 'deep', color='red')

Распределение имеет правостороннюю ассиметрию и островершинность. Даже на глаз видно большое количество выбросов.

In [ ]:
from etna.datasets import TSDataset

In [ ]:
df_etna = df.copy(deep=True)
df_etna['timestamp'] =  pd.to_datetime(df_etna['datetime'])
df_etna["segment"] = df_etna["atm_id"]
df_etna["target"] = df_etna["operation_value"]
df_etna.drop(columns=["operation_value", "atm_id","datetime"], inplace=True)

df_etna.head()

In [ ]:
ts = TSDataset(df_etna, freq="D")
ts

In [ ]:
ts.info()

In [ ]:
ts.describe()

In [ ]:
ts.plot(n_segments = 20)

Почти во всех представленных рядах отслеживаются аномалии (как положительные, так и отрицательные). В большинстве из рядов визуально тренд отсутствует (предположительно имеется позитивный тренд у рядов 103, 104 и 106). Можно предположить, что для всех рядов наблюдается сезонность. Для более подробного анализа посмотрим на статистики рядов:

In [ ]:
plot_correlation_matrix(ts, method="spearman", vmin=0.3, vmax=1)

Ряды друг на друга практически не влияют (большинство корреляций между ними меньше 0,3).  Наибольшая корреляция наблдюается между 74 и 85 банкоматом (равна 0.6).

In [ ]:
acf_plot(ts, lags=21, n_segments = 20)

Выводы по графикам автрегрессии:
1) У 74 и 105 банкоматов наблдюается недельная сезонность
2) У 87, 93, 90, 85, 103 и 104 банкоматов авторегрессия остается высокая на протяжении большого количества лагов, что может говорить  том, что этим временным рядам может свойственна модель авторегрессии. Для подтверждения нужно посмотреть график частной автокрреляции.


In [ ]:
df_etna2 = df_etna[df_etna['timestamp'] >= '2017-08-17']
ts2 = TSDataset(df_etna2.dropna(), freq="D")
ts2

In [ ]:
acf_plot(ts2, partial=True,  lags=10)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_acf(df_etna['target'].diff().dropna(), lags=100, ax=ax)
plt.show()

In [ ]:
from etna.analysis import plot_trend
from etna.transforms import LinearTrendTransform

In [ ]:
plot_trend(ts, trend_transform=LinearTrendTransform(in_column="target"))

У рядов 101, 103, 104, 106,74,85,90,93,94 имеется положительный тренд, а у остальных рядов тренд практически отсутствует.

**Сезонность**
Выше по графикам мы сделали предположение о недельной сезонности, проверим нашу гипотезу с помощью декомпозиции STL

In [ ]:
stl_plot(ts=ts, period=52)

Ни у одноного банкомата недельной сезонности нет :(
  При проверке других периодов сезонности, она такжне обнаружена не была

### Задание 2. Работа с пропущенными значениями и выбросами (1 балл)

Проанализируйте, есть ли в данных пропущенные значения, и подберите оптимальный метод их заполнения средствами ETNA. Какие методы вы выберете и почему? Совет: возможно, лучше сохранить заполненные данные в отдельный объект, поскольку некоторые модели заполняют пропуски встроенными методами, и подача на вход "сырых" данных (на языке временных рядов это данные с нерегулярными интервалами) может принести лучший результат.

Импорты всех необходимых компонент произведите самостоятельно.

In [ ]:
ts.info()

Проверьте ряды на наличие выбросов, и очистите их соответствующим образом. Обоснуйте выбор.

Так как сезонности обнаружено не было, заполним пропуски c помощью  moving average  

In [ ]:
from etna.transforms import (TimeSeriesImputerTransform,
                             DensityOutliersTransform,
                             MedianOutliersTransform)
from etna.analysis.outliers import (get_anomalies_density,
                                    get_anomalies_median,
                                    get_anomalies_prediction_interval)
from etna.analysis import (plot_anomalies, plot_anomalies_interactive,
                           plot_backtest, plot_forecast)
from etna.models import ProphetModel

In [ ]:
nans_imputer = TimeSeriesImputerTransform(
    in_column="target", strategy="running_mean"
)

# df_etna_wo_na = nans_imputer.fit_transform(df_etna)
# ts_wo_na = TSDataset(df_etna_wo_na, freq="D")


ts_wo_na = deepcopy(ts)
ts_wo_na.fit_transform([nans_imputer])

In [ ]:
ts_wo_na.info()

Отклонение от медианных значений, рассчитанных по окну

In [ ]:
anomaly_dict = get_anomalies_median(ts, window_size=50)  # задаем размер окна
plot_anomalies(ts, anomaly_dict)

Метод на основе плотности точек

In [ ]:
anomaly_dict = get_anomalies_density(ts, window_size=18, distance_coef=3, n_neighbors=4)
plot_anomalies(ts, anomaly_dict)

Метод на основе предсказаний модели

In [ ]:
anomaly_dict = get_anomalies_prediction_interval(ts, model=ProphetModel, interval_width=0.95)

plot_anomalies(ts, anomaly_dict)

Для идентификации выбросов будем использовать метод на основе плотности точек, так как визуально он лучше определяет выбросы. Для заполнения пропусков будем использовать также скользящее среднее, так как оно более локально определяет нужное значение и не подвержено влиянию значений, сильно отдаленных от рассматриваемого пропуска.

In [ ]:
ts_filled_na = deepcopy(ts)

ts_filled_na.fit_transform([
    DensityOutliersTransform(in_column="target",  window_size=18, distance_coef=3, n_neighbors=4),
    TimeSeriesImputerTransform(strategy='running_mean'),
])

ts_filled_na.plot(n_segments=20)

теперь круто 😎

### Задание 3. Построение Prophet (1.25 балла)

Постройте прогнозы с помощью Prophet и `etna.Pipeline`, под капотом `etna.Pipeline` обучит `ProphetModel` для каждого сегмента в отдельности. После этого оцените качество по SMAPE на кросс-валидации. В качестве горизонта предсказания возьмите 5 дней - этого же горизонта будем придерживаться и в дальнейшем.

Отрисуйте получившийся прогноз.

In [ ]:
from etna.pipeline import Pipeline
from etna.models import ProphetModel
from etna.metrics import SMAPE, MAE
from etna.analysis import plot_backtest

In [ ]:
HORIZON = 5

In [ ]:
model = ProphetModel()
transforms = []
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na, metrics=[SMAPE(), MAE()], n_jobs=5
)

metrics_df.groupby(['segment']).mean()

In [ ]:
plot_backtest(forecast_df, ts_filled_na, history_len=70)

Исходя из статистики по каждому ряду, какой можно сделать вывод о том, как лучше предсказывать итоговое значение? Посчитайте то же самое исходя из MAE.

Cчитала сразу для SWAPE и MAE, лучше считать по SWAPE

### Задание 4. Иерархический временной ряд (2 балла)

Вопрос, поставленный в прошлом задании, тем не менее, естественным образом подводит нас к концепции **иерархического временного ряда** (когда один ряд состоит из других в качестве компонент). Это полезная концепция, которая может встретиться во многих задачах. В этом задании вам предстоит самостоятельно разобраться в деталях ее реализации в ETNA с использованием справочных материалов, предоставляемых библиотекой (как это часто бывает на практике). Они содержатся по следующей ссылке:

https://github.com/etna-team/etna/blob/master/examples/303-hierarchical_pipeline.ipynb

На выходе из первой части задания необходимо получить `TSDataset` с иерархической структурой, а также соответствующий реконсилятор, который позволит собирать искомый ряд из составных компонент. Классы, которые вам понадобятся, импортированы ниже.

In [ ]:
from etna.datasets import HierarchicalStructure
from etna.pipeline import HierarchicalPipeline
from etna.reconciliation import TopDownReconciliator

In [ ]:
segment = list(map(str, df_etna.segment.unique()))

hierarchical_structure = HierarchicalStructure(
    level_structure={"target": segment}, level_names=["target", "segment"]
)

hierarchical_ts = TSDataset(df=df_etna, freq="D", hierarchical_structure=hierarchical_structure)

hierarchical_ts.head()

In [ ]:
hierarchical_ts.current_df_level

TopDownReconciliator не может работать с отрицательными числами, поэтому прибавим константу

In [ ]:
df_etna['target'].min()

In [ ]:
df_etna1 = df_etna.copy()
df_etna1['target'] = df_etna1['target'] + 1926
df_etna1.head()

In [ ]:
segment = list(map(str, df_etna1.segment.unique()))

hierarchical_structure1 = HierarchicalStructure(
    level_structure={"target": segment}, level_names=["target", "segment"]
)

hierarchical_ts1 = TSDataset(df=df_etna1, freq="D", hierarchical_structure=hierarchical_structure1)

hierarchical_ts1.head()

In [ ]:
ahp_reconciliator = TopDownReconciliator(
    target_level="segment", source_level="target", method="AHP", period=6
)

ahp_reconciliator.fit(ts=hierarchical_ts1)
ahp_reconciliator.mapping_matrix.toarray()

Во второй части задания примените найденные на предыдущих этапах преобразования очистку от выбросов уже к иерархическому датасету, и запустите на нем Prophet с MAE на кросс-валидации.

In [ ]:
hierarchical_ts1.fit_transform([
    DensityOutliersTransform(in_column="target",  window_size=18, distance_coef=3, n_neighbors=4),
    TimeSeriesImputerTransform(strategy='running_mean'),
])

In [ ]:
model = ProphetModel()
transforms = []
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df_hierh, forecast_df_hierh, fold_info_df_hierh = pipeline.backtest(
    ts=hierarchical_ts1, metrics=[MAE()], n_jobs=5
)

metrics_df_hierh.groupby(['segment']).mean()

MAE снизилась для каждого сегмента, иерархическая структура имеет место быть

### Задание 5. Построение признаков (1.5 балла)

Вернемся к нашему исходному мультисегментному ряду - теперь поработаем с моделями, которые требуют построения признаков - `ARIMA` и `CatBoost`. Построим для них признаки, и попробуем при помощи них добиться улучшения качества.

Из обязательного:
1) постройте графики автокорреляции и при помощи них обоснуйте выбор лаговых признаков
2) для катбуста включите в признаки результаты STL разложения. STL используем для убирания тренда за счет преобразования на месте.
* Кроме этого, подумайте на экспертном уровне,
    - Какие еще закономерности могут присутствовать и почему?
    - Что из этого кажется более предсказуемым, и почему?
    - Cделайте соответствующие признаки, и снабдите их кратким комментарием.

3) После этого, обучите на получившихся признаках модели. Для `CatBoost` рассмотрите 2 версии мультисегментную и на каждый ряд в отдельности.

Ниже приведены импорты, которые вам точно понадобятся - к ним необходимо добавить те инструменты, которые вы дополнительно решите использовать в анализе.

In [ ]:
from etna.analysis import acf_plot, stl_plot
from etna.ensembles import DirectEnsemble, StackingEnsemble, VotingEnsemble
from etna.models import (CatBoostMultiSegmentModel, CatBoostPerSegmentModel,
                         AutoARIMAModel)
from etna.transforms import STLTransform, LagTransform, SegmentEncoderTransform, DateFlagsTransform

In [ ]:
acf_plot(ts_filled_na, lags=50)

По ACF видно, что пристутствуют статистически значимые автокорелляции с наблюдениями вплоть до 30 лага почти по всем банкоматам. При этом для болльшинства банкоматов наблюдается увеличение коэффициента корреляции на 6,7 лагах

In [ ]:
acf_plot(ts_filled_na, partial = True, lags=50)

По графикам видно, что переменная имеет положительную частную корелляцию с 1, 6 и 7 лагам.

ARIMA

In [ ]:
model = AutoARIMAModel()
lags = LagTransform(in_column="target", lags=[6, 7])

transforms = [lags]
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na, metrics=[SMAPE(), MAE()], n_jobs=5
)

In [ ]:
metrics_df.groupby(['segment']).mean()

CatBoost

In [ ]:
stl_plot(ts=ts_filled_na, period=7)

In [ ]:
stl = STLTransform(in_column="target", period=7, model="arima")
transformed_train_ts = deepcopy(ts_filled_na)
transformed_train_ts.fit_transform([stl])

In [ ]:
plot_anomalies(
    ts=transformed_train_ts,
    anomaly_dict=get_anomalies_density(ts=transformed_train_ts, window_size=5, distance_coef=2.5),
)

In [ ]:
stl = STLTransform(in_column="target", period=7, model="arima")
model = CatBoostMultiSegmentModel()
anomaly = DensityOutliersTransform(in_column="target", window_size=5, distance_coef=2.5)
seg = SegmentEncoderTransform()

transforms = [stl, anomaly, seg]
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na, metrics=[SMAPE(), MAE()], n_jobs=5
)

metrics_df.groupby(['segment']).mean()

CatBoost для каждого сегмента

In [ ]:
stl = STLTransform(in_column="target", period=7, model="arima")
model = CatBoostPerSegmentModel()
anomaly = DensityOutliersTransform(in_column="target", window_size=5, distance_coef=2.5)
seg = SegmentEncoderTransform()

transforms = [stl, anomaly, seg]
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na, metrics=[SMAPE(), MAE()], n_jobs=5
)

metrics_df.groupby(['segment']).mean()

Тут я затупила и попыталась написать катбуст для каждого сегмента руками из мультисегментного, забавы ради оставлю))

In [ ]:
# import numpy as np

# stl = STLTransform(in_column="target", period=7, model="arima")
# model = CatBoostMultiSegmentModel()
# anomaly = DensityOutliersTransform(in_column="target", window_size=5, distance_coef=2.5)
# seg = SegmentEncoderTransform()

# transforms = [stl, anomaly, seg]
# pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

# metrics = pd.DataFrame({'segment':df_etna.segment.unique(),
#                        'metrics':np.nan})

# for segment in df_etna.segment.unique():
#   metrics_df, forecast_df, fold_info_df = pipeline.backtest(
#       ts=ts_filled_na.get_segment(segment), metrics=[SMAPE(), MAE()], n_jobs=5
#   )
#   metrics.loc[df['segment'] = segment]['metrics'] = metrics_df.mean()

# metrics

Метрики получились относительно высокими (относительно той же профет модели),можно еще попробовать добавить праздничные дни в качестве признаков, наверняка они влияют

In [ ]:
holidays = [1,2,3,4,5,6,7, # new year
            54, # 23 feb
            67, 68, # 8 mar
            121,122,123,124,125,126,127,128,129,130, #may holidays
            163,164, #russia day
            308,309# день народного единства
           ]

In [ ]:
stl = STLTransform(in_column="target", period=7, model="arima")
model = CatBoostMultiSegmentModel()
anomaly = DensityOutliersTransform(in_column="target", window_size=5, distance_coef=2.5)
seg = SegmentEncoderTransform()
d_flags = DateFlagsTransform(day_number_in_year=holidays)

transforms = [stl, anomaly, d_flags, seg]
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na, metrics=[SMAPE(), MAE()], n_jobs=5
)

metrics_df.groupby(['segment']).mean()

Профет все равно лучше

### Задание 6. Стратегии прогнозирования (1.25 балла)

Как нам известно по семинарским занятиям, при построении прогноза на горизонте дальше следующего наблюдения перед нами встает задача определиться со стратегией генерации прогноза. Попробуйте разные стратегии на нашем горизонте из 5 дней (прямая, рекурсивная). Какая стратегия в совокупности с какой моделью дает лучший результат?

Учитывайте особенности, которые некоторые стратегии могут накладывать на признаки (в частности, на лаговые переменные).

При необходимости, в смешанном методе пользуйтесь упрощенной схемой спецификации моделей/преобразований.

In [ ]:
from etna.pipeline import AutoRegressivePipeline, Pipeline, assemble_pipelines
from etna.ensembles import DirectEnsemble

In [ ]:
stl = STLTransform(in_column="target", period=7, model="arima")
anomaly = DensityOutliersTransform(in_column="target", window_size=5, distance_coef=2.5)
lags = LagTransform(in_column="target", lags=[1, 6, 7])
d_flags = DateFlagsTransform(day_number_in_year=holidays)
seg = SegmentEncoderTransform()

AutoRegressivePipeline

In [ ]:
model = ProphetModel()

transforms = [lags, anomaly, d_flags]

autoregressivepipeline = AutoRegressivePipeline(
    model=model, transforms=transforms, horizon=HORIZON, step=1
)

metrics_recursive_df, forecast_recursive_df, _ = autoregressivepipeline.backtest(
    ts=ts_filled_na,
    metrics=[SMAPE(), MAE()],
    n_jobs=3,
    n_folds=3
)

arp_metrics_prophet = metrics_recursive_df.groupby(['segment']).mean()
arp_metrics_prophet['method'] = 'ARP'
arp_metrics_prophet['Model'] = 'Prophet'

In [ ]:
model = CatBoostMultiSegmentModel()

transforms = [stl, anomaly, d_flags, seg]

autoregressivepipeline = AutoRegressivePipeline(
    model=model, transforms=transforms, horizon=HORIZON, step=1
)

metrics_recursive_df, forecast_recursive_df, _ = autoregressivepipeline.backtest(
    ts=ts_filled_na,
    metrics=[SMAPE(), MAE()],
    n_jobs=3,
    n_folds=3
)

arp_metrics_cat = metrics_recursive_df.groupby(['segment']).mean()
arp_metrics_cat['method'] = 'ARP'
arp_metrics_cat['Model'] = 'Catboost'

Pipeline

In [ ]:
model = ProphetModel()

lags_pipe = LagTransform(in_column="target", lags=[6,7])

transforms = [lags_pipe, anomaly, d_flags]

pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na,
    metrics=[SMAPE(), MAE()],
    n_jobs=3,
    n_folds=3
)

pipe_metrics_prophet = metrics_df.groupby(['segment']).mean()
pipe_metrics_prophet['method'] = 'Pipeline'
pipe_metrics_prophet['Model'] = 'Prophet'

In [ ]:
model = CatBoostMultiSegmentModel()

transforms = [stl, anomaly, d_flags, seg]

pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na,
    metrics=[SMAPE(), MAE()],
    n_jobs=3,
    n_folds=3
)

pipe_metrics_cat = metrics_df.groupby(['segment']).mean()
pipe_metrics_cat['method'] = 'Pipeline'
pipe_metrics_cat['Model'] = 'Catboost'

In [ ]:
pipe_metrics_cat['method'][0]

In [ ]:
df_metrics = [arp_metrics_prophet, arp_metrics_cat, pipe_metrics_prophet, pipe_metrics_cat]
res = pd.DataFrame()

for df in df_metrics:
    meth = df['method'][0]
    model = df['Model'][0]
    res[f'SMAPE_{meth}_{model}'] = df['SMAPE']
    # res[f'MAE_{meth}_{model}'] = df['MAE']

In [ ]:
res

Почему-то для Catboost метрики получились одинаковые для обоих методов, а для Prophet лучше использовать Pipeline

### Задание 7. Ансамбли (1.25 балла)

Чтобы не выбирать лучшую модель, мы можем использовать преимущества каждой - давайте объединим все наши наработки в ансамбль. Попробуйте различные опции ансамблирования, и выберите ту, которая работает лучше всего (не забывайте, что внутри каждого ансамбля у нас тоже есть параметры, которые мы можем варьировать - веса в voting, включаемые признаки в stacking).

In [ ]:
from etna.ensembles import StackingEnsemble, VotingEnsemble

In [ ]:
model = SeasonalMovingAverageModel(window=30, seasonality=7)

transforms = []

pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)

metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts_filled_na,
    metrics=[SMAPE(), MAE()],
    n_jobs=5,
    n_folds=5
)

metrics_df.groupby(['segment']).mean()

In [ ]:
plot_backtest(forecast_df, ts_wo_na, history_len=70)

В принципе SeasonalMovingAverageModel хорошо себя показывает исходя из метрик, поэтому ее тоже включим в ансамбль

In [ ]:
stl = STLTransform(in_column="target", period=7, model="arima")
anomaly = DensityOutliersTransform(in_column="target", window_size=5, distance_coef=2.5)
lags = LagTransform(in_column="target", lags=[6, 7])
d_flags = DateFlagsTransform(day_number_in_year=holidays)
seg = SegmentEncoderTransform()

In [ ]:
catboost_pipeline = Pipeline(
    model=CatBoostMultiSegmentModel(),
    transforms=[stl, anomaly, d_flags, seg],
    horizon=HORIZON,
)

seasonalma_pipeline = Pipeline(
    model=SeasonalMovingAverageModel(window=30, seasonality=7),
    transforms=[],
    horizon=HORIZON,
)

prophet_pipeline = Pipeline(
    model=ProphetModel(),
    transforms=[lags, anomaly, d_flags],
    horizon=HORIZON,
)

pipeline_names = ["catboost", "moving average", "prophet"]
pipelines = [catboost_pipeline, seasonalma_pipeline, prophet_pipeline]

In [ ]:
voting_ensemble = VotingEnsemble(pipelines=pipelines, weights=[1, 3, 6], n_jobs=4)

voting_ensamble_metrics, _, _ = voting_ensemble.backtest(
    ts=ts_filled_na,
    metrics=[MAE(), SMAPE()],
    n_folds=5,
    aggregate_metrics=True,
    n_jobs=5,
)

voting_ensamble_metrics

In [ ]:
stacking_ensemble = StackingEnsemble(
    pipelines=pipelines, n_folds=10, n_jobs=4, features_to_use='all'
)

stacking_ensemble_metrics, _, _ = stacking_ensemble.backtest(
    ts=ts_filled_na,
    metrics=[MAE(), SMAPE()],
    n_folds=5,
    aggregate_metrics=True,
    n_jobs=2,
)

stacking_ensemble_metrics

### Задание 8*. Трансформер (бонус 2 балла)

Для желающих протестировать мощь трансформенных моделей предлагается "завести" модель из второго семинара по временным рядам на текущем датасете. На этот раз абсолютно все необходимые импорты и зависимости подгружаются на ваше усмотрение. Удалось ли вам улучшить качество по сравнению с классическими моделями / моделями на основе ML? Что бы вы предпочли внедрить в production?

*NB:* все, конечно, прекрасно знают механизм работы бонусных заданий, но порядка ряди прописать дисклеймер должны - итоговый балл за домашнее задание ставится по формуле min(10, ваш суммарный балл за задания).

In [ ]:
#YOUR CODE HERE#